### Keypoint detection demo

The main point of this notebook demonstrates the details of the Harris keypoint detector.  It ends with an example of using the SIFT class in OpenCV


In [9]:
import cv2         
import numpy as np
from matplotlib import pyplot as plt

In [10]:
'''  We'll test with license plate images from the web '''

# im_name = '../test_images/license_plates/florida-plate.jpg'
im_name = '../examples/wisconsin.jpg'
im = cv2.imread('im_name', cv2.IMREAD_GRAYSCALE)
# ipu.show_with_pixel_values(im)

The next block of code performs the steps of computing the Harris measure.

In [11]:
'''  Gaussian smoothing '''
sigma = 1
ksize = (4*sigma+1,4*sigma+1)
im_s = cv2.GaussianBlur(im.astype(np.float32), ksize, sigma)

'''  Derivative kernels '''
kx,ky = cv2.getDerivKernels(1,1,3)
kx = np.transpose(kx/2)
ky = ky/2

'''  Derivatives '''
im_dx = cv2.filter2D(im_s,-1,kx)
im_dy = cv2.filter2D(im_s,-1,ky)

''' Components of the outer product '''
im_dx_sq = im_dx * im_dx
im_dy_sq = im_dy * im_dy
im_dx_dy = im_dx * im_dy

''' Convolution of the outer product with the Gaussian kernel
    gives the summed values desired '''
h_sigma = 2*sigma
h_ksize = (4*h_sigma+1,4*h_sigma+1)
im_dx_sq = cv2.GaussianBlur(im_dx_sq, h_ksize, h_sigma)
im_dy_sq = cv2.GaussianBlur(im_dy_sq, h_ksize, h_sigma)
im_dx_dy = cv2.GaussianBlur(im_dx_dy, h_ksize, h_sigma)

''' Compute the Harris measure '''
kappa = 0.004
im_det = im_dx_sq * im_dy_sq - im_dx_dy * im_dx_dy
im_trace = im_dx_sq + im_dy_sq
im_harris = im_det - kappa * im_trace*im_trace

''' Show the gray scale image and the Harris image '''

AttributeError: 'NoneType' object has no attribute 'astype'

Now we come to the peak detection and non-maximum suppression step.  This requires use of a combination of thresholding, dilation, pixel-by-pixel comparison, and image multiplication.  See if you can understand the following steps.

In [ ]:
''' Renormalize the intensities into the 0..255 range '''
i_min = np.min(im_harris)
i_max = np.max(im_harris)
print("Before normalization the minimum and maximum harris measures are",
     i_min, i_max)
im_harris = 255 * (im_harris - i_min) / (i_max-i_min)

'''
Apply non-maximum thresholding using dilation, which requires the image
to be uint8.  Comparing the dilated image to the Harris image will preserve
only those locations that are peaks.
'''
max_dist = 2*sigma
kernel = np.ones((2*max_dist+1, 2*max_dist+1), np.uint8)
im_harris_dilate = cv2.dilate(im_harris.astype(np.uint8), kernel)
im_harris[np.where(im_harris < im_harris_dilate)] = 0


'''
Get the normalized Harris measures of the peaks
'''
peak_values = im_harris[np.where(im_harris>0)]
peak_values = np.sort(peak_values, axis=None)
print("After NMS, there are %d peaks and this is %.2f of the total number of pixels"
      % (len(peak_values), len(peak_values)/im_harris.size))
print("And, the top, middle and bottom values are",
      peak_values[-1], peak_values[len(peak_values)//2], peak_values[0])

'''
Keep only those that are in the top fraction.
'''
threshold_frac = 0.25
threshold_index = int((1-threshold_frac)*len(peak_values))
threshold = peak_values[threshold_index]
print("The threshold to keep only the top %.1f percent of the peaks is %.1f"
      % (threshold_frac*100, threshold))
_, im_harris_thresh = cv2.threshold(im_harris.astype(np.uint8), threshold,
                                 1, cv2.THRESH_BINARY)


As the last step we need to extract the Harris keypoints and convert them to OpenCV KeyPoint objects.  Once we do this, we can display the keypoints on the original image. 

In [ ]:
'''  Extract all indices '''
indices = np.where(im_harris_thresh > 0)
ys,xs = indices[0],indices[1]   # rows and columns

''' Put them into the keypoint list '''
kp_size = 4*sigma
harris_keypoints = [
    cv2.KeyPoint(xs[i], ys[i], kp_size)
    for i in range(len(xs))
]

print("We have", len(harris_keypoints), "keypoints.")

out_img = cv2.drawKeypoints( im.astype(np.uint8), harris_keypoints, None)


#### About SIFT: 

While SIFT has been incorporated into OpenCV, it is a patented algorithm.  Therefore, it is not distributed in the pre-built versions of OpenCV, not even of the opencv-contrib-python I suggested in the syllabus that you install. The only way to get SIFT with OpenCV is to build from scratch with modified compiler settings.  I am choosing to not require each student in the class to do this.

The SIFT patent may have already expired, or will expire soon.  Therefore, the non-use of SIFT for class will end when a pre-built 

#### ORB instead:

Following the publication of SIFT, especially the 2004 IJCV version, there was an explosion of research on alternative algorithms. One of these, ORB, was developed and published in 2011 at ICCV, by the original designers of OpenCV. We will use this in class and in your homework to match images and stitch them together. We will not consider too much the details of how ORB works, but will employ the fact that it does produce both keypoint locations and descriptors. For now, we will only look at the ORB detection results.  Many other keypoint and descriptor algorithms have been contributed to OpenCV as can be seen in the class hierarchy for cv::Feature2D https://docs.opencv.org/4.1.1/d0/d13/classcv_1_1Feature2D.html

In [ ]:
im_name = '../test_images/license_plates/wisconsin-plate.jpg'
im = cv2.imread(im_name, cv2.IMREAD_GRAYSCALE)

num_features = 1000
orb = cv2.ORB_create(num_features)        # See method doc for other parameters
kp, des = orb.detectAndCompute(im, None)  # The None argument is where a binary mask could be

print("There are %d keypoints and descriptors, and each descriptor is length %d"
      % (len(kp), len(des[0])))

print('\nHere the first 10 keypoints')
for k in kp[:10]:
    print(k.response, k.pt, k.angle, k.size)

out_im = cv2.drawKeypoints(im, kp, None)
